In [318]:
import tensorflow as tf
from tensorflow.python.framework import ops
from tensorflow.python.ops import gen_nn_ops

@ops.RegisterGradient("GuidedRelu")
def _GuidedReluGrad(op, grad):
    return tf.select(0. < grad, gen_nn_ops._relu_grad(grad, op.outputs[0]), tf.zeros(grad.get_shape()))

#if __name__ == '__main__':



KeyError: "Registering two gradient with name 'GuidedRelu' !(Previous registration was in <module> /Users/xiangli/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3)"

In [319]:
from tensorflow.python.framework import ops
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import control_flow_ops
from tensorflow.python.ops import linalg_ops
from tensorflow.python.ops import math_ops


In [221]:
@ops.RegisterGradient("blah1")
def _blah(op, grad):
    return tf.select(0. < grad, tf.ones(grad.get_shape()), tf.zeros(grad.get_shape()))

In [302]:
with tf.Session() as sess:
    g = tf.get_default_graph()
    x = tf.constant([10., 2.])
    with g.gradient_override_map({'Relu': 'blah1'}):
        y = tf.nn.relu(x)
        z = tf.reduce_sum(y ** 2)
    tf.initialize_all_variables().run()

    print x.eval(), y.eval(), z.eval(), tf.gradients(z, x)[0].eval()

[ 10.   2.] [ 10.   2.] 104.0 [ 1.  1.]


In [34]:

tf.reset_default_graph()

In [299]:
with tf.Session() as sess:
    g = tf.get_default_graph()
    x = tf.constant([10., 2.])
    
    y = tf.nn.relu(x)
    z = tf.reduce_sum(-y ** 2)
    tf.initialize_all_variables().run()

    print x.eval(), y.eval(), z.eval(), tf.gradients(z, x)[0].eval()

[ 10.   2.] [ 10.   2.] -104.0 [-20.  -4.]


In [196]:

@ops.RegisterGradient("gradtest")
def _GradientTest(op, grad_e, grad_v):
    """Gradient for SelfAdjointEigV2 without dependence on eigenvalues"""
    e = op.outputs[0]
    v = op.outputs[1]
    # a = op.inputs[0], which satisfies
    # a[...,:,:] * v[...,:,i] = e[...,i] * v[...,i]
    
    # I do not care about e for my function....
    with ops.control_dependencies([grad_e.op, grad_v.op]):
        if grad_v is not None:
            f = array_ops.matrix_set_diag(
              math_ops.inv(array_ops.expand_dims(e, -2) - array_ops.expand_dims(e, -1)), array_ops.zeros_like(e))
        
            grad_a = math_ops.batch_matmul(v,
              math_ops.batch_matmul(f * math_ops.batch_matmul(v, grad_v, adj_x=True),
              v,adj_y=True))
    return grad_a 

In [322]:
@ops.RegisterGradient("adsfadfad")
def GradientTest2(op, grad_e, grad_v):
    """Gradient for SelfAdjointEigV2 derived with Joan"""
    e = op.outputs[0]
    v = op.outputs[1]
    #dim = v.get_shape()
    with ops.control_dependencies([grad_e.op, grad_v.op]):
        if grad_v is not None:  
            E = array_ops.diag(e)
            #v_proj = arrary.ops.slice(v, [0,0], [])
            grad_grassman = grad_v - math_ops.batch_matmul(math_ops.batch_matmul(v, grad_v), array_ops.transpose(v))
            grad_a = math_ops.batch_matmul(E, math_ops.batch_matmul(v, array_ops.transpose(grad_grassman))
                                           +math_ops.batch_matmul(grad_grassman, array_ops.transpose(v)))
    return grad_a

In [324]:
@ops.RegisterGradient("adfasdfasdfa")
def GradientTest2(op, grad_e, grad_v):
    """Gradient for SelfAdjointEigV2 derived with Joan with grad_stiefel instead of grassman"""
    e = op.outputs[0]
    v = op.outputs[1]
    with ops.control_dependencies([grad_e.op, grad_v.op]):
        if grad_v is not None:  
            E = array.ops.diag(e)
            #v_proj = arrary.ops.slice(v, [0,0], [])
            grad_stiefel = grad_v - math_ops.batch_matmul(math_ops.batch_matmul(v, grad_v), array_ops.transpose(v))
            grad_a = math_ops.batch_matmul(E, math_ops.batch_matmul(v, array_ops.transpose(grad_stiefel))
                                           +math_ops.batch_matmul(grad_stiefel, array_ops.transpose(v)))
    return grad_a 

In [181]:
@ops.RegisterGradient("GradientTest4")
def GradientTest2(op, grad_e, grad_v):
    """Gradient to test vanishing problem"""
    e = op.outputs[0]
    v = op.outputs[1]
    with ops.control_dependencies([grad_e.op, grad_v.op]):
        if grad_v is not None:  
            E = arrary.ops.diag(e)
            #v_proj = arrary.ops.slice(v, [0,0], [])
            grad_grassman = grad_v - math_ops.batch_matmul(math_ops.batch_matmul(v, grad_v), math_ops.transpose(v))
            grad_a = math_ops.batch_matmul(E, math_ops.batch_matmul(v, math_ops.transpose(grad_grassman))
                                           +math_ops.batch_matmul(grad_grassman, math_ops.transpose(v)))
    return grad_a*10000000000000000

In [246]:
@ops.RegisterGradient("adfasdf")
def _adsfadsf(op, grad_e, grad_v):
    """Gradient to test vanishing problem"""
    e = op.outputs[0]
    v = op.outputs[1]
    with ops.control_dependencies([grad_e.op, grad_v.op]):
        if grad_v is not None:  
            grad_a = grad_v
    return grad_a

In [55]:
import numpy as np
import networkx as nx

def balanced_stochastic_blockmodel(communities=2, groupsize=3, p_in=1.0, p_out=0.0):
    #gives dense adjacency matrix representaiton of randomly generated SBM with balanced community size

    G = nx.planted_partition_graph(l=communities, k=groupsize, p_in=p_in, p_out =p_out)
    A = nx.adjacency_matrix(G).todense()
    
    return A



In [146]:

communities = 2 #number of communities, chance to 
group_size = 10 #number of nodes in each communitites (balanced so far)
dim_graph = communities*group_size

A = np.asarray(balanced_stochastic_blockmodel(communities=communities, groupsize=group_size, p_in=0.9, p_out=0.2)).astype(np.double)


In [308]:

with tf.Session() as sess:
    A = [[1,0,0,5], [3,0,0, 4], [5,5,7,0], [0,0,0,6]]
    #np.asarray(balanced_stochastic_blockmodel(communities=communities,
        #                                          groupsize=group_size, p_in=0.9, p_out=0.2)).astype(np.double)
    #g = tf.get_default_graph()
    A = tf.cast(A, dtype = tf.float64)
    r = tf.constant(10.0, dtype = tf.float64)#tf.Variable(tf.random_normal([1], mean=0.0, stddev=10.0, dtype = tf.float64))
    BH = (tf.square(r)-1)*A
    eigenval, eigenvec = tf.self_adjoint_eig(BH)
    eigenvec_proj = tf.slice(eigenvec, [0,0], [communities*group_size, 2])
    #with g.gradient_override_map({'SelfAdjointEigV2': 'GradientTest3'}):
    z = tf.reduce_sum(eigenvec_proj)     
        
    tf.initialize_all_variables().run()

    print z.eval(), tf.gradients(z, r)[0].eval(), tf.gradients(eigenvec_proj, BH)[0].eval(), r.eval()

-0.140883104496 0.0 [[ 0.00269307  0.          0.          0.        ]
 [-0.005119   -0.00124502  0.          0.        ]
 [ 0.00135188  0.00320817 -0.00144805  0.        ]
 [ 0.00145645 -0.00086827 -0.00036872  0.        ]] 10.0


In [323]:

with tf.Session() as sess:
    A = [[1,0,0,5], [3,0,0, 4], [5,5,7,0], [0,0,0,6]]
    #np.asarray(balanced_stochastic_blockmodel(communities=communities,
        #                                          groupsize=group_size, p_in=0.9, p_out=0.2)).astype(np.double)
    g = tf.get_default_graph()
    A = tf.cast(A, dtype = tf.float64)
    r = tf.constant(10.0, dtype = tf.float64)#tf.Variable(tf.random_normal([1], mean=0.0, stddev=10.0, dtype = tf.float64))
    BH = (tf.square(r)-1)*A
    with g.gradient_override_map({'SelfAdjointEigV2': 'adsfadfad'}):
        eigenval, eigenvec = tf.self_adjoint_eig(BH)
        eigenvec_proj = tf.slice(eigenvec, [0,0], [communities*group_size, 2])
        z = tf.reduce_sum(eigenvec_proj)     
        
    tf.initialize_all_variables().run()

    print z.eval(), tf.gradients(z, r)[0].eval(), tf.gradients(eigenvec_proj, BH)[0].eval(), r.eval()

-0.140883104496 -71635.2928372 [[  722.69873403   246.89362202    30.52571667   197.98523214]
 [  163.8596245   -761.25556195  -235.92879104  -288.01770355]
 [  -67.1063394    781.47760725  -630.26948522  -454.05651472]
 [ -909.70732485  1994.00385436  -949.03258805  -632.31125643]] 10.0


In [326]:

with tf.Session() as sess:
    A = [[1,0,0,5], [3,0,0, 4], [5,5,7,0], [0,0,0,6]]
    #np.asarray(balanced_stochastic_blockmodel(communities=communities,
        #                                          groupsize=group_size, p_in=0.9, p_out=0.2)).astype(np.double)
    g = tf.get_default_graph()
    A = tf.cast(A, dtype = tf.float64)
    r = tf.constant(10.0, dtype = tf.float64)#tf.Variable(tf.random_normal([1], mean=0.0, stddev=10.0, dtype = tf.float64))
    BH = (tf.square(r)-1)*A
    with g.gradient_override_map({'SelfAdjointEigV2': 'adfasdfasdfa'}):
        eigenval, eigenvec = tf.self_adjoint_eig(BH)
        eigenvec_proj = tf.slice(eigenvec, [0,0], [communities*group_size, 2])
        z = tf.reduce_sum(eigenvec_proj)     
        
    tf.initialize_all_variables().run()

    print z.eval(), tf.gradients(z, r)[0].eval(), tf.gradients(eigenvec_proj, BH)[0].eval(), r.eval()

 -0.140883104496

LookupError: No gradient defined for operation 'SelfAdjointEigV2_29' (op type: SelfAdjointEigV2)

In [281]:

tf.reset_default_graph()

In [262]:
communities=2
group_size=2
B = np.asarray(balanced_stochastic_blockmodel(communities=communities, groupsize=group_size, p_in=0.9, p_out=0.1)).astype(np.double)
adj = B

In [267]:
A = tf.placeholder(dtype = tf.float64)
B = tf.cast(A, dtype = tf.float64)
communities=2
group_size=2
k = tf.Variable(tf.random_normal([1], mean=0.0, stddev=10.0, dtype = tf.float64))
BH = (tf.square(k)-1)*B
eigenval, eigenvec = tf.self_adjoint_eig(BH)
eigen_proj = tf.slice(eigenvec, [0,0], [communities*group_size, 3])

init = tf.initialize_all_variables()

with tf.Session() as sess:
    g = tf.get_default_graph()
    with g.gradient_override_map({'SelfAdjointEigV2': 'adfasdf'}):
        loss = tf.reduce_sum(eigen_proj) 
         
        optimizer = tf.train.GradientDescentOptimizer(1)
        train = optimizer.minimize(loss)
            
        gradient_l_k = tf.gradients(loss, k)
        gradient_l_bh = tf.gradients(loss, BH)
        #grad_l_eigenvec = tf.gradients(loss, eigenvec_proj)
        sess.run(init)
        
        print sess.run([k, loss, gradient_l_k, gradient_l_bh, eigenvec_proj], feed_dict = {A: adj})

ValueError: Fetch argument <tf.Tensor 'Slice_15:0' shape=(20, 5) dtype=float64> cannot be interpreted as a Tensor. (Tensor Tensor("Slice_15:0", shape=(20, 5), dtype=float64) is not an element of this graph.)

In [268]:
?tf.get_default_graph()

In [271]:
B = np.asarray(balanced_stochastic_blockmodel(communities=communities, groupsize=group_size, p_in=0.4, p_out=0.1)).astype(np.double)
B = tf.cast(B, dtype = tf.float64)
k = tf.Variable(tf.random_normal([1], mean=0.0, stddev=10.0, dtype = tf.float64))
BH = (tf.square(k))*B
eigenval, eigenvec = tf.self_adjoint_eig(BH)


init = tf.initialize_all_variables()

with tf.Session() as sess:

    loss = tf.reduce_sum(eigenvec) 
         
    optimizer = tf.train.GradientDescentOptimizer(1)
    train = optimizer.minimize(loss)
            
    gradient_r = tf.gradients(loss, k)
    gradient_u = tf.gradients(loss, eigenvec)
    sess.run(init)
    for i in range(10):
        sess.run(train)
        print sess.run([k , loss, gradient_r, tf.gradients(eigenvec, BH)])

[array([-10.21789988]), 2.9999999999999996, [array([ -3.54504616e-17])], [array([[  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00],
       [  2.51308517e+14,   2.18909386e-03,   0.00000000e+00,
          0.00000000e+00],
       [ -2.51308517e+14,  -2.39451038e-03,  -4.58360424e-03,
          0.00000000e+00],
       [  7.39833015e-03,   6.77269810e-03,  -6.77269810e-03,
          2.39451038e-03]])]]
[array([-10.21789988]), 2.9999999999999996, [array([ -3.54504616e-17])], [array([[  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00],
       [  2.51308517e+14,   2.18909386e-03,   0.00000000e+00,
          0.00000000e+00],
       [ -2.51308517e+14,  -2.39451038e-03,  -4.58360424e-03,
          0.00000000e+00],
       [  7.39833015e-03,   6.77269810e-03,  -6.77269810e-03,
          2.39451038e-03]])]]
[array([-10.21789988]), 2.9999999999999996, [array([ -3.54504616e-17])], [array([[  0.00000000e+00,   0.00000000e+00,   0.00000000e+00

In [189]:
-1.90287125e-15*1000000000000000

-1.9028712499999998

In [193]:
?tf.gradient_override_map

Object `tf.gradient_override_map` not found.
